In [25]:
import pandas as pd
import numpy as np
from IPython.display import display

# Load and preprocess data
data = pd.read_csv("processed_weather.csv", sep=';', low_memory=False)

cols_to_convert = [
    'MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm',
    'Humidity9am', 'Humidity3pm',
    'WindSpeed9am', 'WindSpeed3pm', 'WindGustSpeed',
    'Pressure9am', 'Pressure3pm'
]
for col in cols_to_convert:
    data[col] = pd.to_numeric(data[col], errors='coerce')

data['temp_day_avg'] = data[['MinTemp', 'MaxTemp', 'Temp9am', 'Temp3pm']].mean(axis=1)
data['humidity_avg'] = data[['Humidity9am', 'Humidity3pm']].mean(axis=1)
data['wind_speed_avg'] = data[['WindSpeed9am', 'WindSpeed3pm', 'WindGustSpeed']].mean(axis=1)
data['pressure_avg'] = data[['Pressure9am', 'Pressure3pm']].mean(axis=1)

# Use only first 500 samples
subProcData = data[['temp_day_avg', 'humidity_avg', 'wind_speed_avg', 'pressure_avg']].iloc[:500].copy()

# Normalize input features to [0,1] for Sugeno inference
subProcData['temp_norm'] = (subProcData['temp_day_avg'] - 0) / (50 - 0)
subProcData['humidity_norm'] = subProcData['humidity_avg'] / 100
subProcData['wind_norm'] = subProcData['wind_speed_avg'] / 100
subProcData['pressure_norm'] = (subProcData['pressure_avg'] - 980) / (60)

# Define weights for Sugeno-style output (can be tuned based on domain knowledge)
w_temp = 0.6
w_hum = 0.4
w_wind = 0.2
w_pres = 0.2

# Sugeno risk score (weighted sum of normalized inputs)
subProcData['Storm Risk Score'] = (
    subProcData['temp_norm'] * w_temp +
    subProcData['humidity_norm'] * w_hum +
    subProcData['wind_norm'] * w_wind +
    (1 - subProcData['pressure_norm']) * w_pres  # inverse pressure
) * 10  # scale to 0-10 like Mamdani

# Categorize score
subProcData['Risk Category'] = subProcData['Storm Risk Score'].apply(
    lambda x: 'Low' if x < 4.8 else ('Medium' if x < 7 else 'High')
)

# Rename columns for display
pretty_df = subProcData.rename(columns={
    'temp_day_avg': 'Temp (°C)',
    'humidity_avg': 'Humidity (%)',
    'wind_speed_avg': 'Wind (km/h)',
    'pressure_avg': 'Pressure (hPa)'
})

pretty_df['Storm Risk Score'] = pretty_df['Storm Risk Score'].round(3)
pretty_df = pretty_df[[
    'Temp (°C)', 'Humidity (%)', 'Wind (km/h)', 'Pressure (hPa)',
    'Storm Risk Score', 'Risk Category']]

print("\n🌪️ Final Storm Risk Prediction (Sugeno):")
display(pretty_df.head(20))



🌪️ Final Storm Risk Prediction (Sugeno):


Temp (°C)  Humidity (%)  Wind (km/h)  Pressure (hPa)  Storm Risk Score  \
0      18.750          46.5    29.333333         1007.40             5.783   
1      18.500          34.5    23.333333         1009.20             5.093   
2      20.700          34.0    30.333333         1008.15             5.512   
3      20.450          30.5    14.666667         1015.20             4.794   
4      24.325          57.5    22.666667         1008.40             6.726   
5      23.450          39.0    33.000000         1007.30             6.124   
6      20.500          34.0    31.333333         1008.90             5.483   
7      19.050          33.5    19.333333         1011.75             4.954   
8      22.525          25.5    38.333333         1006.25             5.615   
9      22.875          42.5    18.000000         1006.35             5.927   
10     23.250          35.0    17.666667         1010.25             5.535   
11     17.625          90.0    19.666667         1007.35             7.197   
12     16.925          84.5    39.000000          993.65             7.736   
13     17.300          54.0    29.333333         1001.50             6.106   
14     18.100          44.5    17.000000         1009.20             5.319   
15     20.250          39.0    36.000000         1011.85             5.648   
16     17.575          75.5    14.000000         1011.30             6.366   
17     18.975          72.5    29.666667         1004.00             6.970   
18     17.550          39.5    28.000000         1009.55             5.261   
19     18.600          35.5    16.333333         1018.15             4.707   

   Risk Category  
0         Medium  
1         Medium  
2         Medium  
3            Low  
4         Medium  
5         Medium  
6         Medium  
7         Medium  
8         Medium  
9         Medium  
10        Medium  
11          High  
12          High  
13        Medium  
14        Medium  
15        Medium  
16        Medium  
17        Medium  
18        Medium  
19           Low